# CNN for Cifar-10 Dataset

## Introduction

In this first notebook we are going to explain how we have set up our work.  
Essentially the process is divided into several parts:
* Set up environment with the Cifar-10 Dataset
* Define a convolutional neural network
* Define a quantization method
* Train the convolutional neural network
* Provide information about CNN's performance and accuracy

### Requirements

For this project we are used:
* Python 3.6
* Tensorflow 1.8.0
* *GPUs parallel calculation manager* nVidia CUDA 9.0
* *GPU-accelerated library* nVidia cuDNN 7.1
* or CPU optimized tensorflow for intel

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import graph_io, graph_util
from tensorflow.tools.graph_transforms import TransformGraph

from cnn.dense import (NET_NAME, dataset_preprocessing_by_keras, eval_fn,
                       forward_pass, loss_fn)
from cnn.model_class import MODELS_DIR, TfClassifier
from cnn.utils.dataset import load_cifar10
from cnn.utils.save_models import load_frozen_graph

C:\Users\Davide\Anaconda3\envs\tfp3.6\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Cifar-10 Dataset

Cifar-10 Dataset is taken from the official website www.cs.toronto.edu.

Dataset is stored in the data directory: cnn/data. From Cifar-10 dataset we are going to take x_train, t_train, x_test and t_test.
The training dataset set is used for training the CNN, the testing dataset is used for evaluate the performance and the accuracy of the network.

In [2]:
x_train, t_train, x_test, t_test = load_cifar10()

In [3]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

It also needs to be normalized

In [4]:
x_train = dataset_preprocessing_by_keras(x_train)
x_train[0, :, :, 0]

array([[-0.68747891, -0.65572952, -0.63985482, ..., -0.95734874,
        -1.00497283, -0.92559935],
       [-0.65572952, -0.63985482, -0.59223073, ..., -0.92559935,
        -1.02084753, -0.86210057],
       [-0.7192283 , -0.63985482, -0.65572952, ..., -0.68747891,
        -0.735103  , -0.70335361],
       ...,
       [-0.36998499, -0.7509777 , -1.00497283, ..., -0.25886212,
        -0.24298742, -0.14773924],
       [-0.25886212, -0.65572952, -1.19546919, ...,  0.31262694,
        -0.00486698, -0.49698256],
       [-0.27473681, -0.35411029, -1.11609571, ...,  0.75711843,
         0.64599556,  0.28087755]])

## CNN Model and Training

We will use a custom made wrapper for tensorfllow NN training and use

This CNN is called *dense_cnn*. Here we will explain how it is composed.

The CNN is composed by several layers. In the first part there are 2 **convolutional** layers and 2 **pooling** layers (they are alternated), then there are a *flatten* layer followed by a **relu** layer, a *dropout* layer and finally a **softmax** layer.

The network uses a stochastic gradient descent optimizer and a categorical crossentropy loss.  
To judge the performance of our model we are used a MSE metric.

In [5]:
model = TfClassifier(NET_NAME, forward_pass, loss_fn, eval_fn,
                     tf.train.AdamOptimizer())

This network is trained for 50 epochs.

In [6]:
history = model.fit(
    [x_train, t_train],
    batch_size=32,
    validation_split=0.1,
    epochs=20,
    verbosity=1,
    keep_prob=0.5)

print(history)

For training: tensorboard --logdir=/tmp/log-tb/dense_cnn/training
For validation: tensorboard --logdir=/tmp/log-tb/dense_cnn/validation
{'accuracy': ((0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002), (0.102, 0.1002)), 'loss': (nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan)}


Then it's evaluated

In [7]:
evals = model.evaluate([x_test, t_test])

print(evals)

INFO:tensorflow:Restoring parameters from C:/Users/Davide/Desktop/CNN/aca8/cnn/models/dense_cnn/model.ckpt
[{'logits': array([[-2.1997044e+02, -3.2214462e+02, -2.7670618e+02, ...,
        -6.3116058e+02, -3.6231104e+02, -1.5103554e+02],
       [-2.5991422e+02, -2.3590515e+02, -5.2669214e+02, ...,
         1.3220389e+02, -3.2022485e+02, -5.7595203e+01],
       [-6.0920856e+01, -3.4712967e+02, -2.1218668e+02, ...,
        -6.7703345e+02, -4.1647598e-01, -8.3027029e+00],
       ...,
       [-1.2296566e+02, -2.8109644e+02, -8.8609543e+01, ...,
        -1.8662666e+02, -2.2918500e+02, -1.8214465e+02],
       [-6.4391699e+02, -5.5609705e+02, -1.2604508e+03, ...,
        -1.4577312e+03, -9.8258875e+02, -4.5983090e+02],
       [ 8.3894096e+01, -3.4208698e+02, -1.5641927e+02, ...,
        -4.8119287e+02, -1.5576271e+02,  9.5194466e+01]], dtype=float32), 'classes': array([6, 7, 8, ..., 5, 9, 9], dtype=int64), 'probabilities': array([[1.3341164e-37, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00

## Optimization Transform

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import graph_io, graph_util
from tensorflow.tools.graph_transforms import TransformGraph

from cnn.dense import (NET_NAME, dataset_preprocessing_by_keras, eval_fn,
                       forward_pass, loss_fn)
from cnn.model_class import MODELS_DIR, TfClassifier
from cnn.utils.dataset import load_cifar10
from cnn.utils.save_models import load_frozen_graph

model = TfClassifier(NET_NAME, forward_pass, loss_fn, eval_fn,
                     tf.train.AdamOptimizer())

In [9]:
opt_graph = model.optimize_for_inference()
opt_graph.get_operations()

INFO:tensorflow:Restoring parameters from C:/Users/Davide/Desktop/CNN/aca8/cnn/models/dense_cnn/model.ckpt
INFO:tensorflow:Froze 8 variables.
Converted 8 variables to const ops.


[<tf.Operation 'logits/bias' type=Const>,
 <tf.Operation 'logits/kernel' type=Const>,
 <tf.Operation 'dense/bias' type=Const>,
 <tf.Operation 'dense/kernel' type=Const>,
 <tf.Operation 'flatten/Reshape/shape/1' type=Const>,
 <tf.Operation 'flatten/strided_slice/stack_2' type=Const>,
 <tf.Operation 'flatten/strided_slice/stack_1' type=Const>,
 <tf.Operation 'flatten/strided_slice/stack' type=Const>,
 <tf.Operation 'conv2d_1/bias' type=Const>,
 <tf.Operation 'conv2d_1/kernel' type=Const>,
 <tf.Operation 'conv2d/bias' type=Const>,
 <tf.Operation 'conv2d/kernel' type=Const>,
 <tf.Operation 'features' type=Placeholder>,
 <tf.Operation 'conv2d/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d/MaxPool' type=MaxPool>,
 <tf.Operation 'conv2d_1/Conv2D' type=Conv2D>,
 <tf.Operation 'conv2d_1/BiasAdd' type=BiasAdd>,
 <tf.Operation 'conv2d_1/Relu' type=Relu>,
 <tf.Operation 'max_pooling2d_1/MaxPool' type=MaxPo